In [24]:
import pandas as pd 
import numpy as np
import scipy.stats as sts
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import chi2_contingency

In [3]:
df = pd.read_csv("new_data.csv")

# Гипотеза № 1:
Судя по графику зависимости стоимости от позиции игрока самыми дорогими игроками выступают игроки нападения. Можно предположить, что игроки нападения в среднем стоят дороже игроков защиты.
\begin{aligned}
& H_0: \mu_{attack} = \mu_{deff} \\
& H_1: \mu_{attack} > \mu_{deff}
\end{aligned}

По ЦПТ средняя стоимость игроков нападения и средняя стоимость игроков защиты распределены нормально, следовательно их разность тоже распределена нормально. Значит можно использовать z статистику для проверки гипотез.

Уровень значимости alpha = 0.1

In [11]:
alpha = 0.1

Находим среднее значение и стандартное отклонение среднего значения стоимости для игроков линии атаки и защиты.

In [20]:
attack_mean = df[df["attack"] == 1].value.mean()
deff_mean = df[df["deff"] == 1].value.mean()
diff = attack_mean - deff_mean
attack_std = df[df["attack"] == 1].value.var(ddof=1) / df[df["attack"] == 1].value.size
deff_std = df[df["deff"] == 1].value.var(ddof=1) /  df[df["deff"] == 1].value.size
diff_std = attack_std + deff_std

Находим z статистику и pvalue для полученного значения статистики.

In [21]:
z_obs = diff / np.sqrt(diff_std)

In [22]:
pvalue = 1 - sts.norm.cdf(z_obs)

In [23]:
pvalue > alpha

False

# Вывод:
Нулевая гипотеза отвергается в пользу альтернативной. Т.е. можно говорить о том, что средняя стоимость игрока нападения выше средней стоимости игрока защиты.

# Гипотеза № 2:
Считается, что игроки защиты  более грубо играют чем остальные, т.е. получают больше красных и желтых карточек. Проверим есть ли зависимость в количестве  карточек от позиции на поле(по одному закону распределения получают карточки игроки защиты и игроки остальных позиций или нет.)
\begin{aligned}
& H_0: F{deff} = F_{non.deff} \\
& H_1: F_{deff} \neq  F_{non.deff}
\end{aligned}

Уровень значимости alpha = 0.1

In [43]:
alpha = 0.1

Списки названий столбцов с карточками и матчами.

In [29]:
card = ["yellow20", "double_yellows20", "red20", "yellow21", "double_yellows21", "red21", "yellow22", "double_yellows22", "red22"]
matches = ["matches20", "matches21", "matches22"]

Делим игроков на защитников и остальных.

In [30]:
deff_players = df[df["deff"] == 1]
non_deff_players = df[df["deff"] != 1]

Находим количество матчей, в которых игроки получали карточки и не получали.(за 1 матч игрок може получить только 1 раз одинарную желтую либо двойную желтую либо прямую красную).

In [31]:
deff_players["matches with card"] = deff_players[card].sum(axis=1)
deff_players["matches without card"] = (deff_players[matches].sum(axis=1) - deff_players[card].sum(axis=1))
non_deff_players["matches with card"] = non_deff_players[card].sum(axis=1)
non_deff_players["matches without card"] = non_deff_players[matches].sum(axis=1) - non_deff_players[card]

Находим общую сумму значений для каждой группы игроков(защитники и не защитники).

In [32]:
deff_card = deff_players["matches with card"].sum()
deff_nocard = deff_players["matches without card"].sum()
non_deff_card = non_deff_players["matches with card"].sum()
non_deff_nocard = non_deff_players["matches without card"].sum()

Заполняем матрицу и выполняем тест хи квадрат независимости.

In [42]:
data_hi = np.array([[deff_card, deff_nocard], [non_deff_card, non_deff_nocard]])
sts.chi2_contingency(data_hi).pvalue > alpha

False

# Вывод:
Полученное значение pvalue меньше заявленного уровня значимости. Следовательно нулевая гипотеза отвергается, и можно говорить о том, что защитник игрок или не защитник влияет на получение им карточки.

# Гипотеза № 3:
Считается, что игроки полузащиты ниже чем остальные, т.к. их в их задачи на поле не входит борьба за мяч в воздухе. Можно предположить, что игроки полузащиты в среднем ниже игроков остальных позиций.
\begin{aligned}
& H_0: \mu_{middle} = \mu_{nonmiddle} \\
& H_1: \mu_{middle} < \mu_{nonmiddle}
\end{aligned}

По ЦПТ средний рост игроков нполузащит и средний рост игроков остальных линий распределены нормально, следовательно их разность тоже распределена нормально. Значит можно использовать z статистику для проверки гипотез.

Уровень значимости alpha = 0.1

In [55]:
alpha = 0.1

Разделим изначальную выборку на игроков полузащиты и остальных.

In [56]:
middle = df[df["middle"] == 1]
nonmiddle = df[df["middle"] == 0]

Находим среднее значение и стандартное отклонение среднего значения роста для игроков линии защиты и остальных.

In [57]:
middle_mean = middle["height"].mean()
nonmiddle_mean = nonmiddle["height"].mean()
middle_std = middle["height"].var(ddof=1) / middle["height"].size
nonmiddle_std = nonmiddle["height"].var(ddof=1) / nonmiddle["height"].size

Находим z статистику и pvalue для полученного значения статистики.

In [58]:
diff = nonmiddle_mean - middle_mean
diff_std = np.sqrt(middle_std + nonmiddle_std)
z_obs = diff / diff_std
pvalue = 1 - sts.norm.cdf(z_obs)

In [59]:
pvalue > alpha

False

# Вывод:
Итоговое значение pvalue не превосходит уровня значимости. Следовательно нулевая гипотеза отвергается. Т.е можно говорить о том, что в среднем полузащитники ниже остальных игроков.